In [ ]:
'''Um projeto de previsão de demanda e otimização de estoque pode ser muito útil para empresas que desejam melhorar sua eficiência operacional,
reduzir custos de armazenamento e evitar a falta ou excesso de produtos.'''

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX



In [ ]:
df = pd.read_csv('demand_inventory (1).csv')
df

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')


In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.set_index('Date', inplace=True)

In [ ]:
df

In [ ]:
# Selecionando os dados dentro do período desejado
periodo_selecionado = df.loc['2023-06-01':'2023-08-01']

# Calculando as vendas por mês
vendas_por_mes = periodo_selecionado.resample('M').sum()
vendas_por_mes


In [ ]:
# Plotando a série temporal de demanda
plt.figure(figsize=(10, 6))
plt.plot(df.index, df['Demand'], color='blue')
plt.title('Demanda ao longo do tempo')
plt.xlabel('Data')
plt.ylabel('Demanda')
plt.grid(True)
plt.show()


In [ ]:
# Selecionando os dados no período especificado
periodo_selecionado = df.loc['2023-06-01':'2023-06-30']

# Ordenando as vendas em ordem decrescente
top_datas_vendas = periodo_selecionado.sort_values(by='Demand', ascending=False)
top_datas_vendas # retornará as datas com as maiores vendas

In [ ]:
df.describe() # dados analiticos

In [ ]:
# Aplicando suavização exponencial para identificar tendências de longo prazo
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Ajustando o modelo de suavização exponencial aos dados de vendas
modelo_suavizacao = ExponentialSmoothing(df['Demand'], trend='add')
resultado_suavizacao = modelo_suavizacao.fit()

# Prevendo as vendas usando o modelo ajustado
previsao_vendas = resultado_suavizacao.fittedvalues

# Plotando as vendas originais e as vendas suavizadas
plt.figure(figsize=(10, 6))
plt.plot(df.index, df['Demand'], label='Vendas Originais', color='blue')
plt.plot(df.index, previsao_vendas, label='Vendas Suavizadas', color='red')
plt.title('Vendas Originais vs. Vendas Suavizadas')
plt.xlabel('Data')
plt.ylabel('Vendas')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Plotando a série temporal de demanda
plt.figure(figsize=(10, 6))
plt.plot(df.index, df['Inventory'], color='blue')
plt.title('Inventário ao longo do tempo')
plt.xlabel('Data')
plt.ylabel('Inventário')
plt.grid(True)
plt.show()


In [ ]:
# Aplicando suavização exponencial para identificar tendências de longo prazo
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Ajustando o modelo de suavização exponencial aos dados de vendas
modelo_suavizacao = ExponentialSmoothing(df['Inventory'], trend='add')
resultado_suavizacao = modelo_suavizacao.fit()

# Prevendo as vendas usando o modelo ajustado
previsao_vendas = resultado_suavizacao.fittedvalues

# Plotando as vendas originais e as vendas suavizadas
plt.figure(figsize=(10, 6))
plt.plot(df.index, df['Inventory'], label='Inventário Original', color='blue')
plt.plot(df.index, previsao_vendas, label='Inventário Suavizado', color='red')
plt.title('Inventário Original vs. Inventário Suavizado')
plt.xlabel('Data')
plt.ylabel('Inventário')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
vendas_por_produto = df.groupby('Product_ID')['Demand'].sum()
print("Vendas Totais por Produto:")
print(vendas_por_produto)


In [ ]:
# Calculando as vendas cruzadas entre produtos
vendas_cruzadas = df.groupby('Product_ID')['Demand'].sum()
print("Vendas Cruzadas entre Produtos:")
print(vendas_cruzadas)


In [ ]:
df['Percentual_Demanda_vs_Inventario'] = (df['Demand'] / df['Inventory']) * 100
df['Percentual_Demanda_vs_Inventario'] 

In [ ]:
# Definindo o período da média móvel (por exemplo, 30 dias)
periodo_media_movel = 30

# Calculando a média móvel das vendas
df['Sales_Moving_Average'] = df['Demand'].rolling(window=periodo_media_movel, min_periods=1).mean()

# Exibindo as primeiras linhas do DataFrame resultante
df.head()

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt

# Ajustando o modelo de Holt-Winters aos dados de vendas
modelo_hw = ExponentialSmoothing(df['Demand'], trend='add', seasonal='add', seasonal_periods=12)
resultado_hw = modelo_hw.fit()

# Prevendo as vendas usando o modelo ajustado
previsao_vendas_hw = resultado_hw.fittedvalues

# Plotando as vendas originais e as vendas previstas
plt.figure(figsize=(10, 6))
plt.plot(df.index, df['Demand'], label='Vendas Originais', color='blue')
plt.plot(df.index, previsao_vendas_hw, label='Vendas Previstas (Holt-Winters)', color='red')
plt.title('Vendas Originais vs. Vendas Previstas (Holt-Winters)')
plt.xlabel('Data')
plt.ylabel('Vendas')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Supondo que você tenha uma coluna 'Inventory' em seu DataFrame df
# Adicione a coluna de inventário ao conjunto de dados
df['Inventory_Available'] = df['Inventory']

# Exibindo as primeiras linhas do DataFrame resultante
df.head()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Vou assumir que você já tem um DataFrame 'data' onde 'Date' é o índice
# Vou supor que 'Demand' é a coluna que queremos analisar

# Se 'Date' já for o índice, você não precisa definir novamente, apenas use-o como está
time_series = df['Demand']

# Diferenciar a série temporal para torná-la estacionária (remover tendência)
differenced_series = time_series.diff().dropna()

# Plot ACF e PACF da série temporal diferenciada
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
plot_acf(differenced_series, ax=axes[0])
plot_pacf(differenced_series, ax=axes[1])
plt.show()


In [ ]:
# Selecionar apenas as colunas numéricas
df_numeric = df.select_dtypes(include=[np.number])

# Calcular a correlação
correlacao = df_numeric.corr()
correlacao


In [ ]:
# Defina order e seasonal_order de acordo com a ordem do seu modelo SARIMA
order = (1, 1, 1)
seasonal_order = (1, 1, 1, 2)

# Crie e ajuste o modelo SARIMA
model = SARIMAX(time_series, order=order, seasonal_order=seasonal_order)
model_fit = model.fit(disp=False)

# Defina quantos passos futuros você deseja prever
future_steps = 30

# Faça previsões para os próximos 'future_steps' passos
predictions = model_fit.predict(len(time_series), len(time_series) + future_steps - 1)
predictions = predictions.astype(int)

print(predictions)

In [ ]:
import numpy as np

# Suponha que você tenha as previsões reais de demanda para agosto de 2023
previsoes_demandas = [
    117, 116, 130, 114, 128, 115, 129, 115, 129, 115, 
    129, 115, 129, 115, 129, 115, 129, 115, 129, 115, 
    129, 116, 130, 116, 130, 116, 130, 116, 130, 116
]

# Define os parâmetros
inventario_inicial = 5500
lead_time = 7
nivel_de_servico = 0.95
custo_de_estoque = 0.1
custo_de_falta_de_estoque = 10

# Calcula a quantidade ótima de pedido usando a fórmula do Newsvendor
z = np.abs(np.percentile(previsoes_demandas, 100 * (1 - nivel_de_servico)))
quantidade_do_pedido = np.ceil(np.mean(previsoes_demandas) + z).astype(int)

# Calcula o ponto de reposição
ponto_de_reposicao = np.mean(previsoes_demandas) * lead_time + z

# Calcula o estoque de segurança ótimo
estoque_de_seguranca = ponto_de_reposicao - np.mean(previsoes_demandas) * lead_time

# Calcula o custo total de manutenção de estoque
custo_total_de_manutencao = custo_de_estoque * (inventario_inicial + 0.5 * quantidade_do_pedido)

# Calcula o custo total de falta de estoque
custo_total_de_falta_de_estoque = custo_de_falta_de_estoque * np.maximum(0, np.mean(previsoes_demandas) * lead_time - inventario_inicial)

# Calcula o custo total
custo_total = custo_total_de_manutencao + custo_total_de_falta_de_estoque

print("Quantidade ótima de pedido:", quantidade_do_pedido)
print("Ponto de reposição:", ponto_de_reposicao)
print("Estoque de segurança:", estoque_de_seguranca)
print("Custo total:", custo_total)


In [ ]:
import numpy as np

# Suponha que você tenha as previsões reais de demanda para agosto de 2023
previsoes_demandas = [
    117, 116, 130, 114, 128, 115, 129, 115, 129, 115, 
    129, 115, 129, 115, 129, 115, 129, 115, 129, 115, 
    129, 116, 130, 116, 130, 116, 130, 116, 130, 116
]

# Define os parâmetros
inventario_inicial = 5500
lead_time = 7
nivel_de_servico = 0.95
custo_de_estoque = 0.1
custo_de_falta_de_estoque = 10

# Calcula a quantidade ótima de pedido usando a fórmula do Newsvendor
z = np.abs(np.percentile(previsoes_demandas, 100 * (1 - nivel_de_servico)))
quantidade_do_pedido = np.ceil(np.mean(previsoes_demandas) + z).astype(int)

# Calcula o ponto de reposição
ponto_de_reposicao = np.mean(previsoes_demandas) * lead_time + z

# Calcula o estoque de segurança ótimo
estoque_de_seguranca = ponto_de_reposicao - np.mean(previsoes_demandas) * lead_time

# Calcula o custo total de manutenção de estoque
custo_total_de_manutencao = custo_de_estoque * (inventario_inicial + 0.5 * quantidade_do_pedido)

# Calcula o custo total de falta de estoque
custo_total_de_falta_de_estoque = custo_de_falta_de_estoque * np.maximum(0, np.mean(previsoes_demandas) * lead_time - inventario_inicial)

# Calcula o custo total
custo_total = custo_total_de_manutencao + custo_total_de_falta_de_estoque

print("Quantidade ótima de pedido:", quantidade_do_pedido)
print("Ponto de reposição:", ponto_de_reposicao)
print("Estoque de segurança:", estoque_de_seguranca)
print("Custo total:", custo_total)
